In [137]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import tomli as tomllib
from transit import transit_validation_2019_alfaro_filepath
from transit_function import read_transit_assignments, read_dbf_and_groupby_sum
from bart import process_BART_data, process_BART_county, process_BART_SL
from muni import process_muni
from screen import concat_final_SL
from map_data import process_muni_map, process_bart_map
from obs import process_obs_data
from total_val import process_valTotal_Operator, process_valTotal_Submode, assign_ferry_name, assign_muni_name
from simwrapper_table import process_mkd_muni, process_mkd_bart, process_mkd_screenline, process_data

In [79]:

with open("transit.toml", "rb") as f:
    config = tomllib.load(f)

model_run_dir = Path(config["directories"]["model_run"])
markdown_output_dir = Path(config["directories"]["markdown_output_dir"])
transit_input_dir = Path(config["directories"]["transit_input_dir"])
observed_BART = Path(config["transit"]["observed_BART"])
observed_BART_county = Path(config["transit"]["observed_BART_county"])
observed_BART_SL = Path(config["transit"]["observed_BART_SL"])
observed_MUNI_Line = Path(config["transit"]["observed_MUNI_Line"])
observed_SL = Path(config["transit"]["observed_SL"])
observed_NTD = Path(config["transit"]["observed_NTD"])
model_BART = Path(config["output"]["model_BART"])
model_BART_county = Path(config["output"]["model_BART_county"])
model_BART_SL = Path(config["output"]["model_BART_SL"])
model_MUNI_Line = Path(config["output"]["model_MUNI_Line"]) 
model_SL = Path(config["output"]["model_SL"])
output_dir = model_run_dir / "validation_workbook" / "output"
output_transit_dir = output_dir / "transit"
test_NTD  = Path(config["transit"]["test_NTD"])
time_periods = ["EA", "AM", "MD", "PM", "EV"]
tod_order = ['EA', 'AM', 'MD', 'PM', 'EV', 'Total']


In [232]:
def NTD_table(transit_input_dir, observed_NTD):
    obs_NTD_df = pd.read_csv(transit_input_dir / observed_NTD)
    obs_NTD_df = obs_NTD_df[["operator", 'annual_upt', 'average weekday_upt']]
    obs_NTD_df['average weekday_upt'] = obs_NTD_df.apply(
        lambda row: round(row['annual_upt'] / 260) if pd.isna(row['average weekday_upt']) else row['average weekday_upt'], axis=1
    )
    obs_NTD_avgupt = obs_NTD_df[['operator', 'average weekday_upt']]
    obs_NTD_avgupt = obs_NTD_avgupt.rename(columns={'operator': 'Operator', 'average weekday_upt': 'Observed'})
    return obs_NTD_avgupt
def calcualte_weekday_UPT(transit_input_dir, observed_NTD):
    obs_NTD_df = pd.read_csv(transit_input_dir / observed_NTD)
    obs_NTD_avgupt = NTD_table(transit_input_dir, observed_NTD)
    AC_Transbay = obs_NTD_df[obs_NTD_df['operator'] == 'AC-Transit']['commuter_bus_total'] / 261
    AC_Eastbay = obs_NTD_df[obs_NTD_df['operator'] == 'AC-Transit']['bus_total'] / 261
    AC = {
        "Operator": ["AC Transbay", "AC Eastbay"],
        "Observed": [AC_Transbay.mean(), AC_Eastbay.mean()],
    }
    df_AC = pd.DataFrame(AC)

    GGT_bus = obs_NTD_df[obs_NTD_df['operator'] == 'GG Transit']['bus_total'] / 261
    GGT_Ferry = obs_NTD_df[obs_NTD_df['operator'] == 'GG Transit']['ferry_total'] / 261
    GG = {
        "Operator": ["GGT-Bus", "GGT-Ferry"],
        "Observed": [GGT_bus.mean(), GGT_Ferry.mean()],
    }
    GG_Transit = pd.DataFrame(GG)

    MUNI_Bus = obs_NTD_df[obs_NTD_df['operator'] == 'MUNI']['bus_total'] / 261
    MUNI_Rail = obs_NTD_df[obs_NTD_df['operator'] == 'MUNI']['light_rail_total']  / 261
    MUNI_Cable = obs_NTD_df[obs_NTD_df['operator'] == 'MUNI']['cable_car_total']  / 261
    MUNI = {
        "Operator": ["MUNI-Bus", "MUNI-Rail", "MUNI_Cable"],
        "Observed": [MUNI_Bus.mean(), MUNI_Rail.mean(), MUNI_Cable.mean()],
    }
    MUNI = pd.DataFrame(MUNI)

    SCVTA_bus = obs_NTD_df[obs_NTD_df['operator'] == 'SCVTA']['bus_total'] / 261
    SCVTA_rail = obs_NTD_df[obs_NTD_df['operator'] == 'SCVTA']['light_rail_total'] / 261
    SCVTA = {
        "Operator": ["SCVTA-Bus", "SCVTA-LRT"],
        "Observed": [SCVTA_bus.mean(), SCVTA_rail.mean()],
    }
    SCVTA = pd.DataFrame(SCVTA)

    # All observed data
    total = pd.concat([obs_NTD_avgupt, df_AC, GG_Transit, MUNI, SCVTA], ignore_index=True)
    df_filtered = total.loc[~total["Operator"].isin(["MUNI", "GG Transit", "SCVTA"])]
    df_filtered = df_filtered.sort_values(by="Operator", ascending=True)
    return df_filtered

name_mapping = {
    "AC Transit": "AC-Transit",
    "Caltrain": "CalTrain",
    "FAST": "FAST",
    "LAVTA-Wheels": "LAVTA",
    "SF MUNI": "MUNI",
    "MUNI_Cable": "MUNI-Cable",
    "Golden Gate Transit": "GG Transit",
    "Napa Vine": "VINE",
    "Petaluma Transit": "Petaluma",
    "SF Bay Ferry": "SF Bay Ferry (WETA)",
    "Santa Rosa CityBus": "Santa Rosa",
    "Tri-Delta": "Tri Delta Transit",
    "Union City": "Union City Transit",
    "Vacaville City Coach": "Vacaville City Coach",
    "WestCAT": "WestCat",
    "Vacaville": "Vacaville City Coach",
    "Shuttle": "Standford Shuttles",
    "Muni-Bus": "MUNI-Bus",
	"Muni-rail": "MUNI-Rail",
	"Muni-Cable": "MUNI-Cable",
}

service_operator_dict = {
    "Premium": ["AC Transbay", "ACE", "Amtrak", "CalTrain", "GGT-Bus", "SMART"],
    "Local Bus": [
        "AC Eastbay", "CCCTA", "EmeryGoRound", "FAST", "LAVTA", "MUNI-Bus", 
        "Petaluma", "Rio Vista Delta Breeze", "SamTrans", "Santa Rosa", 
        "SCVTA-Bus", "SolTrans", "Sonoma County Transit", "Standford Shuttles", 
        "Tri Delta Transit", "Union City Transit", "Vacaville City Coach", 
        "VINE", "WestCat"
    ],
    "BART": ["BART"],
    "Light Rail": ["MUNI-Rail", "MUNI-Cable", "SCVTA-LRT"],
    "Ferry": ["GGT-Ferry", "SF Bay Ferry (WETA)"],
    None: ["AC-Transit"]
}


def process_total_val(dbf_file):
    
    model_operator = read_dbf_and_groupby_sum(dbf_file, None, ["SYSTEM"], "AB_BRDA")
    model_operator = model_operator.groupby("SYSTEM")["AB_BRDA"].sum().reset_index()
    model_operator = model_operator.rename(columns={"SYSTEM": "Operator", "AB_BRDA": "Modeled"})

    all_mode = read_dbf_and_groupby_sum(dbf_file, None, ["MODE"], "AB_BRDA")
    all_mode = all_mode.groupby("MODE")["AB_BRDA"].sum().reset_index()
    all_mode = all_mode.rename(columns={"MODE": "Operator", "AB_BRDA": "Modeled"})
    
    muni_df = read_dbf_and_groupby_sum(dbf_file, "SF MUNI","VEHTYPE", "AB_BRDA" )
    muni_df["MUNI_name"] = muni_df["VEHTYPE"].apply(assign_ferry_name)
    muni = muni_df.groupby("MUNI_name")["AB_BRDA"].sum().reset_index()

    
    ferry_df = read_dbf_and_groupby_sum(dbf_file,'Ferry', 'NAME', 'AB_BRDA')  # List to collect DataFrames
    ferry_df["Ferry_name"] = ferry_df["NAME"].apply(assign_ferry_name)
    ferry = ferry_df.groupby("Ferry_name")["AB_BRDA"].sum().reset_index()

    ac = read_dbf_and_groupby_sum(dbf_file, "AC Transit", "MODE", "AB_BRDA")
    
    GG_bus_model = all_mode[all_mode["Operator"] == 23]["Modeled"].mean()
    VTA_LR_model = all_mode[all_mode["Operator"] == 21]["Modeled"].mean()
    VTA_bus_model = all_mode[all_mode["Operator"] == 19]["Modeled"].mean()
    model_muni_bus = all_mode[all_mode["Operator"] == 11]["Modeled"].mean()
    model_muni_cable = muni[muni["MUNI_name"] == "Cable Car"]["AB_BRDA"].mean()
    model_muni_rail = all_mode[all_mode["Operator"] == 15]["Modeled"].mean()
    ac_trans = ac[ac["MODE"] == 22]["AB_BRDA"].mean()
    ac_east = ac[ac["MODE"] == 18]["AB_BRDA"].mean()
    model_GG_ferry = ferry[ferry["Ferry_name"] == "GGT-Ferry"]["AB_BRDA"].mean()
    model_SF_Bay = ferry[ferry["Ferry_name"] == "SF Bay Ferry"]["AB_BRDA"].mean()
    
    model_dic = {
        "Operator": [
            "GGT-Ferry",
            "GGT-Bus",
            "SCVTA-Bus",
            "SCVTA-LRT",
            "SF Bay Ferry (WETA)",
            "Muni-Bus",
            "Muni-rail",
            "Muni-Cable",
            "AC Transbay",
            "AC Eastbay",
        ],
        "Modeled": [
            model_GG_ferry,
            GG_bus_model,
            VTA_bus_model,
            VTA_LR_model,
            model_SF_Bay,
            model_muni_bus,
            model_muni_rail,
            model_muni_cable,
            ac_trans,
            ac_east,
        ],
    }
    df_model_dic = pd.DataFrame(model_dic)
    df_modeled = pd.concat([model_operator, df_model_dic])
    df_sf_ferry = pd.DataFrame(
        {"Operator": ["SF Bay Ferry (WETA)"], "Modeled": [model_SF_Bay]})
    model_operator = pd.concat([model_operator, df_sf_ferry])
    model_operator["Operator"] = (
        model_operator["Operator"].map(name_mapping).fillna(model_operator["Operator"])
    )
    df_modeled["Operator"] = (
        df_modeled["Operator"].map(name_mapping).fillna(df_modeled["Operator"])
    )
    return df_modeled, model_operator


In [80]:
combined_gdf = read_transit_assignments(model_run_dir, time_periods)

Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAEA.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAAM.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAMD.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAPM.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAEV.DBF


In [233]:
df = pd.DataFrame([(key, value) for key, values in service_operator_dict.items() for value in values], columns=["Service Type", "Operator"])